$$
\newcommand{\bolde}{\boldsymbol{e}}
\newcommand{\boldh}{\boldsymbol{h}}
\newcommand{\boldp}{\boldsymbol{p}}
\newcommand{\boldr}{\boldsymbol{r}}
\newcommand{\boldt}{\boldsymbol{t}}
\newcommand{\boldq}{\boldsymbol{q}}
\newcommand{\boldM}{\boldsymbol{M}}
\newcommand{\boldP}{\boldsymbol{P}}
\newcommand{\boldR}{\boldsymbol{R}}
\newcommand{\boldT}{\boldsymbol{T}}
\newcommand{\boldQ}{\boldsymbol{Q}}
$$

<a name='hw_lidar_odometry'></a>
# Домашняя работа. Лидарная одометрия<sup>[toc](#_toc)</sup>

<a name='_toc'></a>
# Содержание<sup>[toc](#_toc)</sup>
1. [Задание](#task)
2. [Варианты лидарной одометрии](#lo_versions)
    * [Лидарная одометрия v1.0](#lo_v1)
    * [Лидарная одометрия v2.0](#lo_v2)
    * [Лидарная одометрия v3.0](#lo_v3)
    * [Лидарная одометрия v4.0](#lo_v4)
3. [KITTI](#kitti)
4. [Реализация лидарной одометрии](#lo_impl)
    * [Подготовительные действия](#lo_impl_prep)
    * [Основная работа](#lo_impl_main)
    * [Визуализация результатов](#lo_vis)

<a name='task'></a>
# Задание<sup>[toc](#_toc)</sup>

**Лидарная одометрия** &mdash; это способ оценки перемещения лидара (машины), при котором очередное снятое лидарное облако сопоставляется с предыдущим, чтобы найти относительное перемещение лидара в пространстве. Если говорить более точно, то мы оцениваем, насколько лидар сместился относительно своего предыдущего положения, и через последовательность относительных перемещений лидара восстанавливаем его траекторию в пространстве.

В данной домашней работе предлагается реализовать лидарную одометрию на данных из датасета KITTI. С её помощью требуется определить траекторию движения центрального лидара в пространстве, а также восстановить лидарную карту местности, смержив все лидарные облака воедино.

<a name='lo_versions'></a>
# Варианты лидарной одометрии<sup>[toc](#_toc)</sup>

<a name='lo_v1'></a>
## Лидарная одометрия (v1.0)<sup>[toc](#_toc)</sup>

Пусть к нам пришло очередное лидарное облако $\boldQ = [\boldq_1, \dots, \boldq_{M}]$. Предыдущее лидарное облако обозначим как $\boldP = [\boldp_1, \dots, \boldp_N]$. Требуется найти перемещение $\boldT$ лидара в пространстве относительно его предыдущей позы. Если говорить неформально, то это такой трансформ, который делает облако $\boldQ$ &laquo;похожим&raquo; на $\boldP$:
$$
\boldT \otimes \boldQ \approx \boldP.
$$
Эту задачу решает алгоритм ICP, описанный на лекции. Хорошее описание этого алгоритма также можно найти в документации `opend3d` https://www.open3d.org/docs/release/tutorial/pipelines/icp_registration.html. 

В результате из последовательности лидарных облаков $\boldP_0$, $\boldP_1$, $\dots$, $\boldP_t$ получаем последовательность относительных перемещений лидара $\boldT_{1 \to 0}$, $\boldT_{2 \to 1}$, $\dots$, $\boldT_{i \to (i - 1)}$, $\dots$, $\boldT_{t \to (t - 1)}$. Обозначим через $\boldT_{0 \to \mathrm{world}}$ позу лидара в начальный момент времени относительно интересующей нас системы координат под названием $\mathrm{world}$. Это может быть, например, система координат в референсной точке проекции Меркатора. Ну или же вообще единичный трансформ, тогда мы по сути будем оценивать всю траекторию относительно начального положения лидара. В любом случае теперь мы можем оценить позу центрального лидара в момент времени $t$ относительно интересующей нас системы координат $\mathrm{world}$ следующим образом:
$$
\boldT_{t \to \mathrm{world}} = \boldT_{t \to (t - 1)} \otimes \boldT_{(t - 1) \to \mathrm{world}}.
$$
Рекуррентно раскрывая $\boldT_{(t - 1) \to \mathrm{world}}$, получаем
$$
\boldT_{t \to \mathrm{world}} = \boldT_{t \to (t - 1)} \otimes \dots \otimes \boldT_{i \to (i - 1)} \otimes \dots  \otimes \boldT_{1 \to 0} \otimes \boldT_{0 \to \mathrm{world}}.
$$

Также мы можем взять все найденные положения лидара в пространстве $\{\boldT_{i \to \mathrm{world}}\}_{i=0}^t$ и смержить все облака в единое облако карты $\boldM_t$:
$$
\boldM_t = \sum\limits_{i=0}^t \boldT_{i \to \mathrm{world}} \boldP_t
$$

> <span style="color:blue">**Замечание.**</span> При слиянии облаков стоит разряжать карту, например, с помощью вокселизации, чтобы не было роста объема карты в ситуации, когда машина просто стоит.

<a name='lo_v2'></a>
## Лидарная одометрия (v2.0)<sup>[toc](#_toc)</sup>

У описанного выше алгоритма на практике проявляется основной недостаток &mdash; его низкая точность. Причин этому множество. Например, те же пустые пространства между колец в лидарном облаке, из-за которых при движении лидара значительная часть точек нового очередного облака может просто не попадать в окрестность точек предыдущего облака.


Поэтому рассмотрим небольшое расширение простейшей лидарной одометрии, в которой в каждый момент времени будем поддерживать кэш из $T$ предыдущих облаков, смерженных в карту. При поступлении очередного лидарного облака будем матчить его не к предыдущему, а именно к этой __on-the-fly-карте__. Затем добавляем новое облако в кэш, и попутно удаляем самое старое облако.

Пусть в момент времени $t$ к нам пришло облако $\boldP_t$, и мы каким-то обазом нашли его положение $\boldT_{t \to \mathrm{world}}$. Тогда в после матчинга обновляем on-the-fly-карту:
$$
\boldM_{[t - T + 1, t]} = \sum\limits_{i = t - T + 1}^{t} \boldT_{i \to \mathrm{world}} \otimes \boldP_t.
$$
Затем в момент времени $t + 1$ поступает облако $\boldP_{t + 1}$. Матчим его к облаку $\boldM_{[t - T + 1, t]}$, и находим трансформ $\boldT_{(t + 1) \to \mathrm{world}}$. Теперь добавляем облако $\boldP_{t+1}$ в карту и попутно удаляем облако $\boldP_{t - T + 1}$.


Фактически данный алгоритм действий эквивалентен простейшей лидарной одометрии в случае $T = 1$, т.е. когда в кэше всего одно облако.

<a name='lo_v3'></a>
## Лидарная одометрия (v3.0)<sup>[toc](#_toc)</sup>

Предыдущий алгоритм версии v2 страдает в ситации, когда машина долгое время стоит на месте, ведь в таком случае фактичеси мы придем к тому, что весь кэш будет представелен одним облаком. Затем, когда машина начнёт движение, мы вновь столкнёмся с проблемой пустых пространств между кольцами. С этим над что-то делать.

Довольно простое и логичное решение состоит в том, чтобы заменять облака в кэше не безусловно по мере их поступления, а когда, например, мы переместились относительно предыдущей позы (или любой из поз в кэше) на расстояние, превышающее пороговое значение $d$. 

**Собственно именно эту версию лидарной одометрии и предлагается реализовать**. 

Теперь немного о деталях реализиации:
1. Значение кэша $T$ и порога $d$ должно быть легко задать, чтобы перепрогнать одометрию. При $T = 1$ и $d = 0$ алгоритм должен превращаться простейшую лидарную одомерии версии v1.0.
2. При матчинге использует `point-to-plane ICP registration`. Тут всё придумано за нас, и самый простой способ &mdash; это взять примеры попарного матчинга облаков из [документации `open3d`](https://www.open3d.org/docs/release/tutorial/pipelines/icp_registration.html). Однако для этого нам нужны нормале в __target-облаке__, т.е. в облаке $\boldM_{[t - T + 1, t]}$.
3. Нормали также оцениваем с помощью `open3d`. Перед этим не забываем downsample-ить облака при слиянии &mdash; нам нет смысла в 100 точках в окрестности 5 см. Размер вокселя можно поставить оn 5 до 12.5 см. Радиус рассчета нормалей обычно выбирают в диапазоне 30 см до 50 см.
4. В итоге вокселизируем, считаем нормали, и получаем облако карты, подготовленное к регистрации нового облака лидара.
5. Находим положение нового облака относительно карты, обновляем кэш, пересчитываем карту, ждём следующего облака.

**Что должно быть на выходе?** 
1. Должна быт ьвозможность задать параметр $T$ и $d$ и перепрогнать алгоритм одометрии
2. Алгоритм одометрии должен сохранять найденный позы лидаров на диск. Например, можно сделать как-то так:
    ```
    hw/
      poses_T010_d100.txt  # < Размер кэша 10, d = 100см
      poses_T001_d000.txt  # < Размер кэша 1, d = 0см
    ```
3.  Должна быть ячейка кода, в которой можно указать значения параметров $T$ и $d$. Затем она подгрузит позы из файла (если он есть для таких значений) и смержит все облака согласно этим позам в единое облако. При слиянии карту следует вокселизировать с параметром `voxel_size` от 5см до 12.5см
4.  Затем должна быть ячейка визуализации. Тут также должен быть параметр вокселизации, но уже именно для того, чтобы визуализация не тормозила. Тут `voxel_size` может быть порядка 50см.

Проще говоря, должна быть возможность посмотреть на то, что получилось, и при необходимости перепрогнать

<a name='lo_v4'></a>
## Лидарная одометрия (v4.0)<sup>[toc](#_toc)</sup>

На самом деле можно реализовать лидарную одометрию с фактически неограниченным размером кэша $T = \infty$, т.е. когда мы вообще не удаляем отобранные облака из текущей карты, либо добавляем в неё все облака (проводя через downsampling). На практике основная проблема здесь &mdash; неограниченный рост карты. Если же говорить про наши реалиии в контексте датасета, то сложность матчинга и обновления такого облака будет расти со временем. Но все проблемы решаемы:
1. От неограниченного роста карты спасает умный dump кусков карты на диск, и загрузка с диска, когда потребуется (это если говорить про реалии робота).
2. Также можно реализовать хитрое обновление карты, сложность которого будет зависеть только от размера нового лидарного облака, но не от размера текущей карты.

Если скомбинировать оба подхода, то он окзаывается вполне приемлемым для того, чтобы робот во время проезда восстанаваливал карту местности, попутно локализуя себя в ней. Подобные алгоритмы называются _SLAM (Simultaneous Localization and Mapping)_.

In [ ]:
import typing as T
import os
import numbers
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pykitti
import open3d
import plotly.offline as py
py.init_notebook_mode(connected=True)

# PCL
from pylib.pcl.common.cloud_io import read_point_cloud_o3d, read_point_cloud_xyz
from pylib.pcl.common.transform_point_cloud import transform_point_cloud_xyz
from pylib.pcl.common.convert_point_cloud import convert_point_cloud_o3d_to_xyz
from pylib.pcl.filters.voxel_grid import apply_voxel_grid
from pylib.pcl.filters.radius_outlier_removal import apply_radius_outlier_removal
from pylib.pcl.tools.plotly_visualization import (
    create_plotly_figure,
    plot_cloud,
    apply_min_max_scaling,
    convert_values_to_rgba_tuples_f64,
)

# TRANSFORMS
from pylib.transforms.convert_transform import (
    verify_transform_matrix,
    convert_rotation_matrix_to_quaternion,
    convert_quaternion_to_rotation_matrix,
)

# GEO
from pylib.geo.geo_lla_xyz_converter import GeoLlaXyzConverter
from pylib.geo.geo_position_lla import GeoPositionLLA
from pylib.geo.geo_position_xyz import GeoPositionXYZ

# KITTI
from pylib.kitti.dataset_adaptor import KittiDatasetAdaptor
from pylib.kitti.localization import (
    Localization,
    build_localization,
    build_localizations,
)
from pylib.kitti.convert_localization import (
    convert_localization_to_transform_matrix,
    convert_localizations_to_transform_matrices,
)

<a name='kitti'></a>
# KITTI<sup>[toc](#_toc)</sup>

## Загружаем датасет сцены<sup>[toc](#_toc)</sup>

### Подключаемся к данным на диске (в датасете)<sup>[toc](#_toc)</sup>

In [ ]:
KITTI_DIR_PATH = os.path.abspath('./datasets/KITTI/')

# Указываем датасет для загрузки
RIDE_DATE = '2011_09_26'
DRIVE = '0106'  # Вот тут теперь загружаем

# Загружаем данные. Опционально можно указать диапазон фреймов для загрузки
KITTI_DATASET = pykitti.raw(base_path=KITTI_DIR_PATH, date=RIDE_DATE, drive=DRIVE)

# Сразу же создаем адаптор
KITTI_DATASET_ADAPTOR = KittiDatasetAdaptor(KITTI_DATASET)
NUM_FRAMES = KITTI_DATASET_ADAPTOR.num_lidar_clouds

print('Dataset info:')
print(f'\tdirectory: <{KITTI_DIR_PATH}>')
print(f'\tnum_velodyne_clouds: {NUM_FRAMES}')

### Локализация и позы<sup>[toc](#_toc)</sup>

#### Формируем показания локализации<sup>[toc](#_toc)</sup>

In [ ]:
KITTI_DATASET_ADAPTOR = KittiDatasetAdaptor(KITTI_DATASET)
reference_point = KITTI_DATASET_ADAPTOR.read_geo_positions_lla()[0]
localizations = KITTI_DATASET_ADAPTOR.build_localizations(reference_point)

del reference_point
assert len(localizations) == NUM_FRAMES
print(f'num_localizations: {len(localizations)}')

#### Формируем позы в системе координат меркатора<sup>[toc](#_toc)</sup>

In [ ]:
gps_to_mercator_transforms = convert_localizations_to_transform_matrices(localizations)

### Облака велодайна<sup>[toc](#_toc)</sup>

Чем лучше начальное приближение, тем лучше будет сходимость ICP. У нас есть два простых способа задать начальный трансформ между двумя последовательными лидарными облаками:
1. Единичный трансформ. Скорее всего будет не очень хорошим приближением в случае быстрого движения машины.
2. На основе показаний локализации, где в качестве начального трансформа можно взять преобразование (трансформ) из source-позы в target-позу.

Но довольно слов, давайте просто посмотрим на оба варианта. В ячейке кода ниже мы берем три последовательных облака и отрисовываем их либо как есть, т.е. предполагая единичный трансформ, либо же приводим их в единую систему координат world (относительно которой заданы позы в `poses`). Первый или второй варинат задаются параметром `apply_to_world_transform`.

> **Внимание**. Движение машины начинается только где-то с 50-го фрейма. До этого момента она или практически неподвижна. Ниже специально был подобран фрейм 90, чтобы продемонстрировать разницу между начальными приближениями.

In [ ]:
init_frame_idx = 90

# Используем вокселизация для уменьшения размера облака, чтобы отрисовка не тормозила
voxel_size = 0.15

# Привести к единой системе координат world (на основе показаний GPS-а), или нет?
apply_transform_to_world = True

clouds_xyz = []
for frame_idx in range(init_frame_idx, init_frame_idx + 3):
    cloud_xyz = KITTI_DATASET_ADAPTOR.read_lidar_cloud_xyzi(frame_idx)[..., :3]
    if apply_transform_to_world:
        cloud_xyz = transform_point_cloud_xyz(
            cloud_xyz.astype(np.float64),
            gps_to_mercator_transforms[frame_idx]).astype(np.float32)
    initial_cloud_size = cloud_xyz.shape[0]
    cloud_xyz = apply_voxel_grid(cloud_xyz, voxel_size)
    final_cloud_size = cloud_xyz.shape[0]
    print(f'{frame_idx}: {initial_cloud_size} -> voxel_grid(voxel_size={voxel_size}) -> {final_cloud_size}')
    clouds_xyz.append(cloud_xyz)
    del cloud_xyz

figure = create_plotly_figure(bgcolor='black')
plot_cloud(clouds_xyz[0], colors='red', figure=figure)
plot_cloud(clouds_xyz[1], colors='green', figure=figure)
plot_cloud(clouds_xyz[2], colors='blue', figure=figure)
figure.show()

del clouds_xyz

<a name='lo_impl'></a>
# Реализация лидарной одометрии<sup>[toc](#_toc)</sup>
* [Подготовительные действия](#lo_impl_prep)
* [Основная работа](#lo_impl_main)
* [Визуализация результатов](#lo_vis)

<a name='lo_impl_prep'></a>
## Подготовительные действия<sup>[toc](#_toc)</sup>

У нашей лидарной одометрии два основных параметра:
* `max_clouds_cache_size` &mdash; максимальное количество облаков в текущем кэше
* `min_distance_between_clouds_cm` &mdash; минимальное расстояние нового облака относительно предыдущего, при котором добавляем его в кэш (попутно удаляя самое старое облако из того же кэша). Расстояние указываем в сантиметрах

In [ ]:
# Два основных параметра, описывающих нашу лидарную одометрию
max_clouds_cache_size: int = 10
min_distance_between_clouds_cm: int = 100  # In [cm]eters
del max_clouds_cache_size, min_distance_between_clouds_cm  # Пока удаляем

Всего облаков у нас 227. Поэтому нет смысла указывать размер кэша больше, чем количество облаков:

In [ ]:
MAX_CLOUDS_CACHE_SIZE = KITTI_DATASET_ADAPTOR.num_lidar_clouds

### Сохранение и загрузка результатов<sup>[toc](#_toc)</sup>

Результаты будем хранить в отдельно директории `results` в следующем формате:
```
{current workding dir}/
  results/
    lidar_trajectory_1_100.txt
    lidar_trajectory_10_200.txt
    ...
```

In [ ]:
TRAJECTORIES_DIRPATH = os.path.abspath('results')
TRAJECTORY_FILENAME_PREFIX = 'lidar_trajectory'
TRAJECTORY_FILENAME_EXT = 'txt'


def get_results_filename(max_clouds_cache_size: int, min_distance_between_clouds_cm: int) -> str:
    assert isinstance(max_clouds_cache_size, numbers.Integral)
    assert isinstance(min_distance_between_clouds_cm, numbers.Integral)
    assert max_clouds_cache_size <= MAX_CLOUDS_CACHE_SIZE
    assert max_clouds_cache_size <= MAX_CLOUDS_CACHE_SIZE
    return '{}_{}_{}.{}'.format(
        TRAJECTORY_FILENAME_PREFIX,
        max_clouds_cache_size,
        min_distance_between_clouds_cm,
        TRAJECTORY_FILENAME_EXT)


def parse_results_filename(filename: str) -> T.Tuple[int, int]:
    assert filename.startswith(TRAJECTORY_FILENAME_PREFIX)
    basename, ext = os.path.splitext(filename)
    assert ext == '.' + TRAJECTORY_FILENAME_EXT
    max_clouds_cache_size, min_distance_between_clouds_cm = basename.split('_')[-2:]
    max_clouds_cache_size = int(max_clouds_cache_size)
    min_distance_between_clouds_cm = int(min_distance_between_clouds_cm)
    return max_clouds_cache_size, min_distance_between_clouds_cm


def get_results_filepath(max_clouds_cache_size: int, min_distance_between_clouds_cm: int) -> str:
    return os.path.join(
        TRAJECTORIES_DIRPATH,
        get_results_filename(max_clouds_cache_size, min_distance_between_clouds_cm))


max_clouds_cache_size = 123
min_distance_between_clouds_cm = 456
print('Example of results file for T = {}, d = {} [cm]:'.format(
    max_clouds_cache_size, min_distance_between_clouds_cm))
print(f'\t{get_results_filename(max_clouds_cache_size, min_distance_between_clouds_cm)}')
del max_clouds_cache_size, min_distance_between_clouds_cm

os.makedirs(TRAJECTORIES_DIRPATH, exist_ok=True)
print(f'\nLidar odometry results are stored in <{TRAJECTORIES_DIRPATH}>')
results_filenames = sorted(os.listdir(TRAJECTORIES_DIRPATH))
if len(results_filenames) == 0:
    print('No currently present results')
else:
    print('Currently presents results:')
    for results_filename in sorted(os.listdir(TRAJECTORIES_DIRPATH)):
        max_clouds_cache_size, min_distance_between_clouds_cm = parse_results_filename(results_filename)
        print('\t{} -> T = {}, d = {} [cm]'.format(results_filename, max_clouds_cache_size, min_distance_between_clouds_cm))
    del results_filename, max_clouds_cache_size, min_distance_between_clouds_cm
del results_filenames

Для сохранения найденных поз/трансформов в файл и их считывание будем использовать функции ниже:

In [ ]:
def write_transforms_matrices_to_file(filepath: str, transforms_matrices: T.List[np.ndarray]):
    with open(filepath, 'w') as f:
        f.write('# QX QY QZ QW TX TY TZ TW\n')
        for transform_matrix in transforms_matrices:
            verify_transform_matrix(transform_matrix)
            qx, qy, qz, qw = convert_rotation_matrix_to_quaternion(transform_matrix[:3, :3])
            tx, ty, tz = transform_matrix[:3, 3]
            f.write('{} {} {} {} {} {} {}\n'.format(qx, qy, qz, qw, tx, ty, tz))

def read_transforms_matrices_from_file(filepath: str) -> T.List[np.ndarray]:
    transforms_matrices: T.List[np.ndarray] = []
    with open(filepath, 'r') as f:
        for line in f.readlines():
            if line.startswith('#'):
                continue
            qx, qy, qz, qw, tx, ty, tz = [float(v) for v in line.split()]
            rotation_matrix = convert_quaternion_to_rotation_matrix([qx, qy, qz, qw])
            translation_vector = np.array([tx, ty, tz])
            transform_matrix = np.eye(4, dtype=np.float64)
            transform_matrix[:3, :3] = rotation_matrix
            transform_matrix[:3, 3] = translation_vector
            transforms_matrices.append(transform_matrix)
    return transforms_matrices

<a name='lo_impl_main'></a>
## Основная работа<sup>[toc](#_toc)</sup>

### Находим траекторию лидара<sup>[toc](#_toc)</sup>

Результатом выполняния кода в ячейках данного блока должен быть список `lidar_trajectory` из поз центрального лидара в системе координат, связанной с положением самого превого лидара. Чтобы далее иметь возможность запустить блоки с сохранением, загрузкой и визуализацией результатов, ниже этот список можно инициализировать единичными трансформами или на основе траектории GPS-сенсора в системе меркатора.

Требуется запустить лиданую одометрию для следующих параметров:
1. Лидарная одометрия версии 1.0:
    * `max_clouds_cache_size = 1`
    * `min_distance_between_clouds_cm = 0` &mdash; всегда заменяем предыдущее облако новым
2. Лидарная одометрия версии 2.0:
    * `max_clouds_cache_size = 10`
    * `min_distance_between_clouds_cm = 0` &mdash; всегда вытесняем самое старое облако новым
3. Лидарная одометрия версии 3.0:
    * `max_clouds_cache_size = 10`
    * `min_distance_between_clouds_cm = 100` &mdash; вытесняем самое старое облако, только если сдвинулись отностиельно текущей позиции на $\geqslant 100$ сантиметров
4. То же самое, что и в предыдущем пункте, но теперь для диапазона значений `max_clouds_cache_size` от 1 до 20 (для 10 уже подсчитали в предыдущем пункте). Это нам потребуется чуть дальше, чтобы оценить, насколько лучше становится наша восстановленная карта в зависимости от параметров одометрии. Что касается параметра `min_distance_between_clouds_cm`, то можно либо просто оставить 1 метр, либо поставить что-то свое большее нуля, но главное, чтобы он бы одинаковый для всех размеров кэша.

In [ ]:
# Параметры для запуска одометрии
max_clouds_cache_size: int = 10
min_distance_between_clouds_cm: int = 100  # Внимание, в матрицах трансформов лежат метры. Не забудьте отмасштабировать на 100

In [ ]:
# TODO: Реализовать лидарную одометрию

# Проверка, чтобы случайно не перетереть результаты
results_filepath = get_results_filepath(max_clouds_cache_size, min_distance_between_clouds_cm)
assert not os.path.isfile(results_filepath),\
    'File {os.path.basename(results_filepath} already exists. '\
    'Please disable this check manually or remove file if you want to recalculate odometry'
del results_filepath

In [ ]:
# Для дальшнейшей работы пока инициализируем так
lidar_trajectory = [np.eye(4) for _ in range(NUM_FRAMES)]

# ... ну или, что ещё лучше, так
lidar_trajectory = [np.eye(4)]
for pose_idx in range(1, NUM_FRAMES):
    lidar_trajectory.append(np.dot(
        np.linalg.inv(gps_to_mercator_transforms[0]), gps_to_mercator_transforms[pose_idx]))

### Сохраняем полученную траекторию<sup>[toc](#_toc)</sup>

In [ ]:
# results_filepath = get_results_filepath(max_clouds_cache_size, min_distance_between_clouds_cm)
results_filepath = 'example.txt'
print(f'Saving trajectory to <{results_filepath}>')

write_transforms_matrices_to_file(results_filepath, lidar_trajectory)
del results_filepath

<a name='lo_vis'></a>
## Визуализация результатов<sup>[toc](#_toc)</sup>

<a name='lo_vis_restored_map'></a>
### Визуализация восстановленной карты

В данном блоке просто проверяем себя, что то, что получили, смотрится адекватно

#### Выбираем параметры для визуализации<sup>[toc](#_toc)</sup>

In [ ]:
max_clouds_cache_size: int = 10
min_distance_between_clouds_cm: int = 100

#### Загружаем результаты из файла<sup>[toc](#_toc)</sup>

In [ ]:
# results_filepath = get_results_filepath(max_clouds_cache_size, min_distance_between_clouds_cm)
results_filepath = 'example.txt'
print(f'Loading trajectory from <{results_filepath}>')

lidar_trajectory = read_transforms_matrices_from_file(results_filepath)
del results_filepath
print(f'Loaded {len(lidar_trajectory)} poses/transforms')

#### Переводим облака лидара в единую систему координат<sup>[toc](#_toc)</sup>

In [ ]:
transformed_clouds_xyz = []
for frame_idx in range(NUM_FRAMES):
    cloud_xyz = KITTI_DATASET_ADAPTOR.read_lidar_cloud_xyzi(frame_idx)[..., :3]
    transformed_cloud_xyz = transform_point_cloud_xyz(
        cloud_xyz.astype(np.float64),
        lidar_trajectory[frame_idx]).astype(np.float32)
    transformed_clouds_xyz.append(transformed_cloud_xyz)

##### Создаем единое смерженное облако<sup>[toc](#_toc)</sup>

In [ ]:
voxel_size = 0.5

# Чтобы не перегружать визуализацию, пока ограничим отрисовку
frames_range = range(90, 120)

if frames_range is None:
    frames_range = list(range(NUM_FRAMES))

merged_cloud_xyz = np.vstack([transformed_clouds_xyz[frame_idx] for frame_idx in frames_range])
print(f'Number of points in merged cloud: {merged_cloud_xyz.shape[0]}')

merged_cloud_xyz = apply_voxel_grid(merged_cloud_xyz, voxel_size)
print(f'Number of points after voxelization: {merged_cloud_xyz.shape[0]}')

#### Визуализируем смерженное облако<sup>[toc](#_toc)</sup>

In [ ]:
figure = create_plotly_figure(bgcolor='black')
plot_cloud(merged_cloud_xyz, colors='red', figure=figure)
figure.show()

<a name='lo_vis_restored_map_sizes'></a>
## График размера карты от параметров одометрии

В предыдущем пункте при визуализации мы мержили лидарные облака согласно их позам, и вокселизировали смерженное облако, тем самым получая некоторую карту местности. Причем интуитивно можем определить качество карты, как размер полученного облака, так как чем лучше карта, тем она менее &laquo;пушистая&raquo;. Собственно, **нужно построить график зависимости размера восстановленной карты от размера кэша в одометрии (`max_clouds_cache_size`)**.

In [ ]:
max_clouds_cache_sizes = list(range(1, 20))
voxel_size = 0.125

# TODO: Заполнить список
restored_map_sizes = []
for max_clouds_cache_size in max_clouds_cache_sizes:
    # Вот тут вычисляем размер карты
    restored_map_size = 10000

    restored_map_sizes.append(restored_map_size)
    del max_clouds_cache_size, restored_map_size

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(
    max_clouds_cache_sizes, restored_map_sizes,
    linestyle='-', marker='o', color='b', alpha=0.5)
plt.grid(which='both', linestyle='--', alpha=0.5);